<a href="https://colab.research.google.com/github/askolesov0608/SJK006-Project/blob/main/SJK006_RF_GB_LR_SVM_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler

# Загрузка данных
file_path = 'TestDataCV_DigitalV2.xlsx'
data = pd.read_excel(file_path)

# Создание новой целевой переменной для должности
positions = ['Human Resources', 'Research & Development', 'Sales',
             'Life Sciences', 'Marketing', 'Medical', 'Other']
data['Position'] = (data[positions] == 1).dot(range(1, len(positions) + 1))

# Выбор признаков для модели
features_columns = ['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
                    'HourlyRate', 'JobInvolvement', 'JobLevel', 'GenderM', 'GenderF',
                    'Single', 'Married', 'Divorced']
features = data[features_columns]
target = data['Position']

# Бутстрэппинг каждого класса до 1000 записей
bootstrapped_data = pd.DataFrame(columns=features_columns + ['Position'])
for position in target.unique():
    position_data = data[data['Position'] == position]
    bootstrapped_sample = resample(position_data, replace=True, n_samples=1000, random_state=42)
    bootstrapped_data = pd.concat([bootstrapped_data, bootstrapped_sample])

# Подготовка данных для обучения
X = bootstrapped_data[features_columns]
y = bootstrapped_data['Position'].astype(int)  # Убедитесь, что y - это массив целых чисел

# Масштабирование признаков
X_scaled = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Определение моделей для сравнения
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
    "SVM": SVC(random_state=42)
}


# Обучение и оценка каждой модели
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    print(f"Модель: {name}")
    print(f"Точность: {accuracy}")
    print("Отчет о классификации:")
    print(report)
    print("\n")

Модель: Random Forest
Точность: 0.966
Отчет о классификации:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       265
           1       1.00      1.00      1.00       292
           2       0.98      1.00      0.99       290
           3       1.00      1.00      1.00       320
           4       1.00      1.00      1.00       324
           6       0.91      0.87      0.89       308
           7       0.93      1.00      0.96       318
           8       0.88      0.83      0.85       303
           9       0.97      0.97      0.97       288
          10       1.00      1.00      1.00       292

    accuracy                           0.97      3000
   macro avg       0.97      0.97      0.97      3000
weighted avg       0.97      0.97      0.97      3000



Модель: Gradient Boosting
Точность: 0.8026666666666666
Отчет о классификации:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00  